In [2]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from pprint import pprint
from scipy.optimize import curve_fit
from lmfit.models import ExponentialModel, PowerLawModel, ConstantModel

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import ipympl

In [3]:
SECOND_PUMPDOWN = sio.get_folderpath("20201022_SecondPumpdown")

In [3]:
df1 = sio.read_tpg_data("pressure-monitoring01", SECOND_PUMPDOWN)
df2 = sio.read_tpg_data("pressure-monitoring02", SECOND_PUMPDOWN)

df = pd.concat([df1, df2])

mpl_times = sio.convert_tpg_to_mpl_time(df)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.set_yscale("log")

ax.set_ylabel(r"Pressure (mbar)")

ax.plot(mpl_times, df["Main"], "-", label="Main")
ax.plot(mpl_times, df["Prep"], "-", label="Prep")
ax.legend()

#ax.set_xlim((0, 0.045))
#sio.savefig("pressure-monitoring01_02", SECOND_PUMPDOWN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Extrapolation

Several fits to the whole data were attempted using both Exponentional and PowerLaw models. These fits do not converge (both for scipy and lmfit).
We thus resort to just taking a linear fit from the endpoints of the data.

In [4]:
df = sio.read_tpg_data("pressure-monitoring02", SECOND_PUMPDOWN)

x = sio.convert_tpg_to_mpl_time(df)
y1 = df["Main"]
y2 = df["Prep"]

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.set_ylabel(r"Pressure (mbar)")
#ax.set_yscale("log", base=10)

ax.plot(x, y1, "-", label="Main")
ax.plot(x, y2, "-", label="Prep")
ax.legend()

start = datetime.datetime.strptime("23-Oct-20 12:00", "%d-%b-%y %H:%M")
end = datetime.datetime.strptime("5-Nov-20 12:00", "%d-%b-%y %H:%M")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]
dt_series_mpl = matplotlib.dates.date2num(date_generated)

dfc = df[4000:]
popt, pcov = curve_fit(sft.func_linear, xdata=dfc["MPL_datetimes"], ydata=dfc["Main"])
ax.plot(dt_series_mpl, sft.func_linear(dt_series_mpl, *popt), "--", label="End extrapolation")

popt, pcov = curve_fit(sft.func_linear, xdata=dfc["MPL_datetimes"], ydata=dfc["Prep"])
ax.plot(dt_series_mpl, sft.func_linear(dt_series_mpl, *popt), "--", label="End extrapolation")

#sio.savefig("pressure-monitoring02", SECOND_PUMPDOWN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Temperature stability

In [4]:
df = pd.read_excel(SECOND_PUMPDOWN+"temperature-monitoring01.xls", skiprows=3)
df["Time"] /= 1000 * 60

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
tip_mean = np.mean(df["Tip Holder"])
tip_stddev = 2 * np.std(df["Tip Holder"])

ax1.plot(df["Time"], df["Tip Holder"], "-")
ax1.set_ylabel("Tip Holder (K)")
ax1.axhline(tip_mean, linestyle="--", zorder=3, color="C1", label=f'mean: {tip_mean:.2f} K')
ax1.axhline(tip_mean+tip_stddev, linestyle="--", zorder=3, color="C2", label=f'2-sigma: {tip_stddev:.2f} K')
ax1.axhline(tip_mean-tip_stddev, linestyle="--", zorder=3, color="C2")
ax1.legend()

base_mean = np.mean(df["Baseplate"])
base_stddev = 2 * np.std(df["Baseplate"])

ax2.plot(df["Time"], df["Baseplate"], "-")
ax2.axhline(base_mean, linestyle="--", zorder=3, color="C1", label=f'mean: {base_mean:.3f} K')
ax2.axhline(base_mean+base_stddev, linestyle="--", zorder=3, color="C2", label=f'2-sigma: {base_stddev:.3f} K')
ax2.axhline(base_mean-base_stddev, linestyle="--", zorder=3, color="C2")

ax2.set_ylabel("Baseplate (K)")
ax2.set_xlabel("Time (minutes)")
ax2.legend()

#sio.savefig("temperature-monitoring01", SECOND_PUMPDOWN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
df = sio.read_tpg_data("pressure-monitoring03", SECOND_PUMPDOWN)

x = sio.convert_tpg_to_mpl_time(df)
y1 = df["Main"]
y2 = df["Prep"]

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.set_ylabel(r"Pressure (mbar)")
#ax.set_yscale("log", base=10)

ax.plot(x, y1, "-", label="Main")
ax.plot(x, y2, "-", label="Prep")
ax.legend()

start = datetime.datetime.strptime("26-Oct-20 12:00", "%d-%b-%y %H:%M")
end = datetime.datetime.strptime("5-Nov-20 12:00", "%d-%b-%y %H:%M")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]
dt_series_mpl = matplotlib.dates.date2num(date_generated)

popt, pcov = curve_fit(sft.func_linear, xdata=df["MPL_datetimes"], ydata=df["Main"])
ax.plot(dt_series_mpl, sft.func_linear(dt_series_mpl, *popt), "--", label="End extrapolation")

popt, pcov = curve_fit(sft.func_linear, xdata=df["MPL_datetimes"], ydata=df["Prep"])
ax.plot(dt_series_mpl, sft.func_linear(dt_series_mpl, *popt), "--", label="End extrapolation")

#sio.savefig("pressure-monitoring03_extrapolation", SECOND_PUMPDOWN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# New prep gauge

In [ ]:
df = pd.read_excel(SECOND_PUMPDOWN+"temperature-monitoring01.xls", skiprows=3)
df["Time"] /= 1000 * 60

fig, ax = plt.subplots(nrows=2, sharex=True)
ax.plot(df["Time"], df["Baseplate"], "-")
ax.set_ylabel("Baseplate (K)")
ax.legend()

In [25]:
df = sio.read_tpg_data("pressure-monitoring04", SECOND_PUMPDOWN)
df = df[2150:]

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)

myFmt = DateFormatter("%H:%M\n%a")
ax2.xaxis.set_major_formatter(myFmt)
ax2.set_ylabel(r"Pressure (mbar)")
ax1.set_ylabel(r"Pressure (mbar)")

#ax.set_yscale("log", base=10)

ax1.plot(df["MPL_datetimes"], df["Backing"], "-", label="Backing")
ax2.plot(df["MPL_datetimes"], df["Prep"], "-", label="Prep")

ax1.legend()
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …